In [4]:
###Libraries and imports
import numpy as np
import math
import random
import os
import shutil
import gzip
import glob
import gc
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import tarfile
import PIL
import scipy.misc
import skimage
import nibabel as nib
import pdb
from matplotlib.patches import Rectangle
import pickle
from skimage.metrics import hausdorff_distance

from PIL import Image
from tqdm import tqdm
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras import backend as K
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from skimage.io import imread
from scipy import ndimage as ndi
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (
    Activation,
    Conv3D,
    Conv3DTranspose,
    MaxPooling3D,
    UpSampling3D,
)
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers, regularizers
from skimage.morphology import binary_closing

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

# Path to BraTS2021 folder
data_dir = '/content/drive/My Drive/BraTS2021'

# List all files in the directory
files = os.listdir(data_dir)
print("Files in BraTS2021:", files)


Files in BraTS2021: ['BraTS2021_00495_t1ce.nii.gz', 'BraTS2021_00495_t2.nii.gz', 'BraTS2021_00495_flair.nii.gz', 'BraTS2021_00495_t1.nii.gz', 'BraTS2021_00495_seg.nii.gz']


In [3]:
import nibabel as nib

# File paths
t2_path = os.path.join(data_dir, 'BraTS2021_00495_t2.nii.gz')
t1ce_path = os.path.join(data_dir, 'BraTS2021_00495_t1ce.nii.gz')
t1_path = os.path.join(data_dir, 'BraTS2021_00495_t1.nii.gz')
flair_path = os.path.join(data_dir, 'BraTS2021_00495_flair.nii.gz')
seg_path = os.path.join(data_dir, 'BraTS2021_00495_seg.nii.gz')

# Load images
t2_img = nib.load(t2_path).get_fdata()
t1ce_img = nib.load(t1ce_path).get_fdata()
t1_img = nib.load(t1_path).get_fdata()
flair_img = nib.load(flair_path).get_fdata()
seg_img = nib.load(seg_path).get_fdata()

print("Data Loaded Successfully")
print("T2 shape:", t2_img.shape)
print("T1ce shape:", t1ce_img.shape)
print("T1 shape:", t1_img.shape)
print("FLAIR shape:", flair_img.shape)
print("Segmentation mask shape:", seg_img.shape)


Data Loaded Successfully
T2 shape: (240, 240, 155)
T1ce shape: (240, 240, 155)
T1 shape: (240, 240, 155)
FLAIR shape: (240, 240, 155)
Segmentation mask shape: (240, 240, 155)


In [5]:
# Normalize images
def normalize(image):
    return (image - np.mean(image)) / np.std(image)

In [6]:
t2_img = normalize(t2_img)
t1ce_img = normalize(t1ce_img)
t1_img = normalize(t1_img)
flair_img = normalize(flair_img)

In [8]:
# Stack modalities along the channel axis
multi_modal_img = np.stack([t2_img, t1ce_img, t1_img, flair_img], axis=0)  # Shape: (4, 240, 240, 155)
# Convert segmentation mask to integers
seg_img = seg_img.astype(np.int64)  # Ensure compatibility with PyTorch

print("Multi-modal image shape:", multi_modal_img.shape)
print("Segmentation mask shape:", seg_img.shape)

Multi-modal image shape: (4, 240, 240, 155)
Segmentation mask shape: (240, 240, 155)


In [9]:
import torch
from torch.utils.data import Dataset, DataLoader

class BrainDataset(Dataset):
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks

    def __len__(self):
        return self.images.shape[-1]  # Number of slices

    def __getitem__(self, idx):
        # Return a single slice
        image = self.images[:, :, :, idx]  # Shape: (4, 240, 240)
        mask = self.masks[:, :, idx]      # Shape: (240, 240)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(mask, dtype=torch.long)

# Create dataset and dataloader
dataset = BrainDataset(multi_modal_img, seg_img)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

print("Dataset and DataLoader Prepared")


Dataset and DataLoader Prepared


In [10]:
# Iterate through the DataLoader to fetch batches
for batch_idx, (images, masks) in enumerate(dataloader):
    print(f"Batch {batch_idx}:")
    print("Image batch shape:", images.shape)  # Expected: (Batch_Size, 4, 240, 240)
    print("Mask batch shape:", masks.shape)    # Expected: (Batch_Size, 240, 240)
    break


Batch 0:
Image batch shape: torch.Size([1, 4, 240, 240])
Mask batch shape: torch.Size([1, 240, 240])
